In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE
from sklearn.cross_validation import train_test_split
from math import sqrt
import math
from numpy import mean

In [2]:
train=pd.read_csv('/home/neha/Desktop/jupiter/chemical_original.csv')

In [3]:
#univariate analysis
print train.describe()
print train.dtypes

       BiologicalMaterial01  BiologicalMaterial02  BiologicalMaterial03  \
count            176.000000            176.000000            176.000000   
mean               6.411420             55.688750             67.705000   
std                0.713923              4.034581              4.001064   
min                4.580000             46.870000             56.970000   
25%                5.977500             52.680000             64.980000   
50%                6.305000             55.090000             67.220000   
75%                6.870000             58.737500             70.427500   
max                8.810000             64.750000             78.250000   

       BiologicalMaterial04  BiologicalMaterial05  BiologicalMaterial06  \
count            176.000000            176.000000            176.000000   
mean              12.349261             18.598636             48.910398   
std                1.774661              1.844141              3.746072   
min                9.380

In [4]:
#--> check for missing values(first remove the missing values before bivariate analysis)
print train.apply(lambda x: sum(x.isnull()))

BiologicalMaterial01       0
BiologicalMaterial02       0
BiologicalMaterial03       0
BiologicalMaterial04       0
BiologicalMaterial05       0
BiologicalMaterial06       0
BiologicalMaterial07       0
BiologicalMaterial08       0
BiologicalMaterial09       0
BiologicalMaterial10       0
BiologicalMaterial11       0
BiologicalMaterial12       0
ManufacturingProcess01     1
ManufacturingProcess02     3
ManufacturingProcess03    15
ManufacturingProcess04     1
ManufacturingProcess05     1
ManufacturingProcess06     2
ManufacturingProcess07     1
ManufacturingProcess08     1
ManufacturingProcess09     0
ManufacturingProcess10     9
ManufacturingProcess11    10
ManufacturingProcess12     1
ManufacturingProcess13     0
ManufacturingProcess14     1
ManufacturingProcess15     0
ManufacturingProcess16     0
ManufacturingProcess17     0
ManufacturingProcess18     0
ManufacturingProcess19     0
ManufacturingProcess20     0
ManufacturingProcess21     0
ManufacturingProcess22     1
ManufacturingP

In [8]:
missing=train.apply(lambda x: sum(x.isnull()))
column_name=list(train.columns.values)

for i,var in enumerate(column_name):
	if missing[i]!=0:
		m=mean(train[var])
		train[var].fillna(m,inplace='True')

#-->check again for missing value(remove comment below to check)
#print train.apply(lambda x: sum(x.isnull()))

In [9]:
#bivariate analysis

#--->correlation cofficient between continuous-continuous attribute
#we want to find the correlation between "manufacturing process" and "yield"
x=train.iloc[:,12] 
y=train.iloc[:,-1]  #-->last column 
i=1
print "Correlation between manufacturing process variables and Yield:\n"
for index in range(12,57):
	result=np.corrcoef(train.iloc[:,index],y)
	print "manufacturing Process",i," Yield: ",result[0][1]
	i=i+1
    
    

Correlation between manufacturing process variables and Yield:

manufacturing Process 1  Yield:  -0.100937030519
manufacturing Process 2  Yield:  -0.203495270395
manufacturing Process 3  Yield:  -0.0877739042231
manufacturing Process 4  Yield:  -0.26249432677
manufacturing Process 5  Yield:  0.109925681804
manufacturing Process 6  Yield:  0.387176994606
manufacturing Process 7  Yield:  -0.0431539217217
manufacturing Process 8  Yield:  0.0146365675819
manufacturing Process 9  Yield:  0.503470506954
manufacturing Process 10  Yield:  0.202517562412
manufacturing Process 11  Yield:  0.323597422165
manufacturing Process 12  Yield:  0.348286981922
manufacturing Process 13  Yield:  -0.503679718409
manufacturing Process 14  Yield:  -0.0059462861069
manufacturing Process 15  Yield:  0.216187972072
manufacturing Process 16  Yield:  -0.0373464549193
manufacturing Process 17  Yield:  -0.425806871755
manufacturing Process 18  Yield:  -0.0589292500071
manufacturing Process 19  Yield:  0.134879045229

In [10]:
#scatter plots between input and output variable
plt.scatter(x,y,c=['r','b'])
plt.xlabel('ManufacturingProcess01 (red points)')
plt.ylabel('Yield(blue points)')
plt.title('Manufacturing Process01 v/s yield')
plt.show()


/usr/lib/pymodules/python2.7/matplotlib/collections.py:548: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


In [11]:
#feature selection(we are perfroming feature selection for "manufacturing process" variables)
check_mp=train.iloc[:,range(12,58)]  #select all "manufacturing process" variable

array=check_mp.values
x=array[:,0:45]    # gives values from 0 to 44 index
y=array[:,45]

model=linear_model.BayesianRidge()

rfe=RFE(model,24)
fit=rfe.fit(x,y)
print fit.n_features_
result=fit.support_

print result  #features which got selected by RFE are set to true


24
[ True False False  True False  True  True False  True  True False False
  True False False False  True False False False False  True  True  True
  True False False  True  True False  True  True  True False  True False
  True False  True False False  True  True  True  True]


In [12]:
#add the index of attribute which we want to delete
index=list()
for i,val in enumerate(result):
     if val==False:
        index.append(i)

# because we want to store the first 12 attribute as it is
for i in range(len(index)):
    index[i]=12+index[i]

#drop the un-important attributes
train.drop(train.columns[index],axis=1,inplace=True)

In [13]:
#apply model
#total no. of columns
no_columns=len(train.columns)

#divide the data into train and test set
x_train,x_test,y_train,y_test=train_test_split(train.iloc[:,range(no_columns-1)],train.iloc[:,-1],test_size=0.3,random_state=0)

regr=linear_model.BayesianRidge(n_iter=500)
regr.fit(x_train,y_train)

predicted=regr.predict(x_test)
corr=np.corrcoef(predicted,y_test)
rms = sqrt(mean_squared_error(y_test, predicted))

print "predicted values: ", predicted
print "\n correlation cofficient:\n ",corr[0][1]
print "\n root mean square error: ",rms

predicted values:  [ 41.70742275  39.70803668  39.25053739  40.23178533  40.50085581
  37.84658757  42.83831938  39.08014396  41.1699201   38.31483295
  38.34657147  39.62742093  43.11635859  40.00272573  38.286707
  39.45100447  41.02658435  38.96239892  38.85737945  40.99982983
  41.40674191  43.13022306  42.92637678  40.14892879  40.03236615
  40.57514131  40.07139209  39.38587649  40.40499716  40.3545658
  35.9881061   39.96383241  38.90540335  42.77696398  41.87590762
  42.7112743   39.76999272  39.69226607  38.42084408  40.88291938
  43.00604655  41.47772895  39.21055174  40.38346107  39.1494426
  39.27905444  38.77431203  39.47631214  40.2048771   41.30612587
  38.5194175   39.20682238  42.81841076]

 correlation cofficient:
  0.834959609435

 root mean square error:  0.998823531357


In [15]:
#graph showing relation between actual value and predicted value
actual=np.array(y_test)
x = range(1,len(y_test)+1)
y = actual

x2 = range(1,len(y_test)+1)
y2 = predicted

plt.plot(x, y, label='actual data')
plt.plot(x2, y2, label='Predicted data')

plt.xlabel('number of test data')
plt.ylabel('output range')
plt.title('actual data v/s Predicted data')
plt.legend()
plt.show()